## DataSet

https://huggingface.co/datasets/ProgramComputer/voxceleb

## Model

# Set Up Dependencies

In [40]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -Uq datasets transformers transformers[onnx] ezkl tf-keras librosa "soundfile>=0.12.1" torchaudio

Note: you may need to restart the kernel to use updated packages.


# Test the installation

In [3]:
# Let's see how to retrieve time steps for a model
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC
from datasets import load_dataset
import datasets
import torch

# import model, feature extractor, tokenizer
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = AutoTokenizer.from_pretrained("facebook/wav2vec2-base-960h")
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

# load first sample of English common_voice
dataset = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="train", streaming=True, trust_remote_code=True)
dataset = dataset.cast_column("audio", datasets.Audio(sampling_rate=16_000))
dataset_iter = iter(dataset)
sample = next(dataset_iter)

# forward sample through model to get greedily predicted transcription ids
input_values = feature_extractor(sample["audio"]["array"], return_tensors="pt").input_values
logits = model(input_values).logits[0]
pred_ids = torch.argmax(logits, axis=-1)

# retrieve word stamps (analogous commands for `output_char_offsets`)
outputs = tokenizer.decode(pred_ids, output_word_offsets=True)
# compute `time_offset` in seconds as product of downsampling ratio and sampling_rate
time_offset = model.config.inputs_to_logits_ratio / feature_extractor.sampling_rate

word_offsets = [
    {
        "word": d["word"],
        "start_time": round(d["start_offset"] * time_offset, 2),
        "end_time": round(d["end_offset"] * time_offset, 2),
    }
    for d in outputs.word_offsets
]
# compare word offsets with audio `en_train_0/common_voice_en_19121553.mp3` online on the dataset viewer:
# https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0/viewer/en
word_offsets[:3]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Reading metadata...: 948736it [00:18, 51827.88it/s]
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


[{'word': 'THE', 'start_time': np.float64(0.7), 'end_time': np.float64(0.78)},
 {'word': 'TRICK',
  'start_time': np.float64(0.88),
  'end_time': np.float64(1.08)},
 {'word': 'APPEARS',
  'start_time': np.float64(1.2),
  'end_time': np.float64(1.64)}]

ONNX Version of Wav2Vec2

https://huggingface.co/optimum/wav2vec2-base-superb-sv
https://huggingface.co/optimum/wav2vec2-base-superb-sd

To manually convert model to ONNX

https://huggingface.co/docs/transformers/serialization#exporting-a-model-with-transformersonnx

Use transformers.onnx package as a Python module to export a checkpoint using a ready-made configuration:


In [ ]:
! python -m transformers.onnx --model=facebook/wav2vec2-base-960h onnx/

2024-11-02 03:55:02.647101: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-02 03:55:02.729125: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-02 03:55:02.811940: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730519702.983406   23486 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730519703.058248   23486 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-02 03:55:03.562571: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

: 

# EZKL 

use the EZKL package to run a publicly known / committed to network on some private data, producing a public output.


In [ ]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
from torch import nn
import ezkl
import os
import json
import torch


# Defines the model
# we got convs, we got relu, we got linear layers
# What else could one want ????

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(in_channels=2, out_channels=3, kernel_size=5, stride=2)

        self.relu = nn.ReLU()

        self.d1 = nn.Linear(48, 48)
        self.d2 = nn.Linear(48, 10)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)

        # flatten => 32 x (32*26*26)
        x = x.flatten(start_dim = 1)

        # 32 x (32*26*26) => 32x128
        x = self.d1(x)
        x = self.relu(x)

        # logits => 32x10
        logits = self.d2(x)

        return logits


circuit = MyModel()

# Train the model as you like here (skipped for brevity)

In [ ]:
circuit

MyModel(
  (conv1): Conv2d(1, 2, kernel_size=(5, 5), stride=(2, 2))
  (conv2): Conv2d(2, 3, kernel_size=(5, 5), stride=(2, 2))
  (relu): ReLU()
  (d1): Linear(in_features=48, out_features=48, bias=True)
  (d2): Linear(in_features=48, out_features=10, bias=True)
)

In [7]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [8]:

shape = [1, 28, 28]
# After training, export to onnx (network.onnx) and create a data file (input.json)
x = 0.1*torch.rand(1,*shape, requires_grad=True)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))


In [9]:
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # private by default

res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)

assert res == True


In [10]:
cal_path = os.path.join("calibration.json")

data_array = (torch.rand(20, *shape, requires_grad=True).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

# Serialize data into file:
json.dump(data, open(cal_path, 'w'))


await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")

[tensor] decomposition error: integer -545140602 is too large to be represented by base 16384 and n 2
forward pass failed: "failed to forward: [halo2] General synthesis error"
[tensor] decomposition error: integer -545140602 is too large to be represented by base 16384 and n 2
forward pass failed: "failed to forward: [halo2] General synthesis error"
[tensor] decomposition error: integer -2182662032 is too large to be represented by base 16384 and n 2
forward pass failed: "failed to forward: [halo2] General synthesis error"
[tensor] decomposition error: integer -2182662032 is too large to be represented by base 16384 and n 2
forward pass failed: "failed to forward: [halo2] General synthesis error"
[tensor] decomposition error: integer -8716885742 is too large to be represented by base 16384 and n 2
forward pass failed: "failed to forward: [halo2] General synthesis error"
[tensor] decomposition error: integer -8716885742 is too large to be represented by base 16384 and n 2
forward pass f

True

In [11]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [12]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [13]:
# srs path
res = await ezkl.get_srs( settings_path)

In [14]:
# now generate the witness file

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [15]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,

    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [16]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,

        "single",
    )

print(res)
assert os.path.isfile(proof_path)

{'instances': [['1ce0480000000000000000000000000000000000000000000000000000000000', '1a66f1ef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', '3d6fa0ef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', '937af3ef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', 'f88e580000000000000000000000000000000000000000000000000000000000', '4feb7eef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', '9db0e4ef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', '0328630000000000000000000000000000000000000000000000000000000000', 'b6dbdcef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', 'bb36960000000000000000000000000000000000000000000000000000000000']], 'proof': '0x1e3f6fe4c64b87fc768c12c127746de43b7a5c000cccae15f985edbe5cbb71ef25648eecdd9d5ea8b37d00c617aa79ae8ce3c706c496f186f9ef608aae22cb97002d9f155b4506d624c44f55ab6b14ee0dc1ba693388c787ce9d756841b419f10460f3d5108399eac98f20d59e5332171d314135231d9efaee926c8ea34a300a25e9bcbc95d0956944f7ed41082aef2464

In [17]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,

    )

assert res == True
print("verified")

verified


# EZKL 
https://docs.ezkl.xyz/

## Model 1 : wav2vec2

In [1]:
# Using Wav2Vec with EZKL
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch

model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

# Export to ONNX
torch.onnx.export(model, "wav2vec.onnx")

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-02 07:44:34.628520: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730533474.643587  107667 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730533474.648178  107667 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-02 07:44:34.665831: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical ope

TypeError: string indices must be integers, not 'tuple'

In [4]:
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import torchaudio
import ezkl
from pathlib import Path
import json

class Wav2Vec2EZKL:
    def __init__(self, model_name="facebook/wav2vec2-base"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.processor = Wav2Vec2Processor.from_pretrained(model_name)
        self.model = Wav2Vec2Model.from_pretrained(model_name).to(self.device)
        self.sample_rate = 16000
        self.window_size = 16000
        
        # EZKL paths
        self.model_path = Path("wav2vec2.onnx")
        self.settings_path = Path("settings.json")
        self.compiled_path = Path("model.ezkl")
        self.pk_path = Path("proving.key")
        self.vk_path = Path("verification.key")
        self.proof_path = Path("proof.json")
        self.input_path = Path("input.json")
        
        # Export to ONNX if not exists
        if not self.model_path.exists():
            self.export_to_onnx()

    def export_to_onnx(self):
        dummy_input = torch.randn(1, self.window_size)
        input_names = ["input_values"]
        output_names = ["last_hidden_state"]
        
        torch.onnx.export(
            self.model,
            dummy_input,
            self.model_path,
            input_names=input_names,
            output_names=output_names,
            dynamic_axes={'input_values': {0: 'batch_size'}}
        )

    def preprocess_audio(self, audio_path):
        waveform, sample_rate = torchaudio.load(audio_path)
        if sample_rate != self.sample_rate:
            resampler = torchaudio.transforms.Resample(sample_rate, self.sample_rate)
            waveform = resampler(waveform)
        return waveform.squeeze()
    
    def setup_circuit(self):
        # Generate settings
        if not self.settings_path.exists():
            ezkl.gen_settings(self.model_path, self.settings_path)

            # Calibrate settings
            ezkl.calibrate_settings(self.model_path, self.settings_path, target="resources")

        if not self.compiled_path.exists():
            # Compile circuit
            ezkl.compile_circuit(
                self.model_path, 
                self.compiled_path, 
                self.settings_path
            )
        
        # Generate proving/verification keys
        if not self.pk_path.exists() or not self.vk_path.exists():
            ezkl.setup(
                self.compiled_path,
                self.pk_path,
                self.vk_path,
                self.settings_path
            )

    def create_proof(self, audio_path):
        # Preprocess audio
        waveform = self.preprocess_audio(audio_path)
        inputs = self.processor(waveform, sampling_rate=self.sample_rate, return_tensors="pt")
        input_values = inputs.input_values.numpy().tolist()
        
        # Create input file
        ezkl.create_input_file(input_values, self.input_path)
        
        # Generate proof
        ezkl.prove(
            self.model_path,
            self.pk_path,
            self.proof_path,
            self.settings_path,
            self.input_path
        )
        
        return self.proof_path.exists()

    def verify_proof(self):
        return ezkl.verify(
            self.proof_path,
            self.vk_path,
            self.settings_path
        )

    def run_workflow(self, audio_path):
        self.setup_circuit()
        proof_generated = self.create_proof(audio_path)
        verification_status = self.verify_proof()
        
        return {
            "proof_generated": proof_generated,
            "verification_status": verification_status
        }

# Usage:
import soundfile as sf
audio_info = sf.info("test.wav")
print(audio_info)

pipeline = Wav2Vec2EZKL()
results = pipeline.run_workflow("test.wav")

test.wav
samplerate: 44100 Hz
channels: 2
duration: 2.879 s
format: WAV (Microsoft) [WAV]
subtype: Signed 16 bit PCM [PCM_16]


/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/configuration_utils.py:306: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


RuntimeError: Failed to generate settings: [graph] [tract] Undetermined symbol in expression: sequence_length

## Model 2 : Whisper

In [1]:
import whisper
import torch
import torchaudio
import ezkl
from pathlib import Path
from transformers import WhisperProcessor, WhisperForConditionalGeneration

class WhisperEZKL:
    def __init__(self, model_name="openai/whisper-tiny"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.processor = WhisperProcessor.from_pretrained(model_name)
        self.model = WhisperForConditionalGeneration.from_pretrained(model_name)
        self.sampling_rate = 16000
        
        # EZKL paths
        self.model_path = Path("whisper.onnx")
        self.settings_path = Path("settings.json")
        self.compiled_path = Path("circuit.ezkl")
        self.pk_path = Path("proving.key")
        self.vk_path = Path("verification.key")
        self.proof_path = Path("proof.json")
        self.input_path = Path("input.json")
        
        # Export to ONNX if not exists
        if not self.model_path.exists():
            self.export_to_onnx()

    def export_to_onnx(self):
        dummy_input = torch.randn(1, 80, 3000)
        dummy_tokens = torch.zeros(1, 448, dtype=torch.long) 

        input_names = ["input_values"]
        output_names = ["last_hidden_state"]
        
        torch.onnx.export(
            self.model,
            (dummy_input, dummy_tokens),
            self.model_path,
            input_names=["input_features", "decoder_input_ids"],
            output_names=["logits"],
            dynamic_axes={
                "input_features": {0: "batch", 2: "sequence"},
                "decoder_input_ids": {0: "batch", 1: "sequence"}
            }
        )

    def preprocess_audio(self, audio_path):
        waveform, sample_rate = torchaudio.load(audio_path)
        if sample_rate != self.sample_rate:
            resampler = torchaudio.transforms.Resample(sample_rate, self.sample_rate)
            waveform = resampler(waveform)
        return waveform.squeeze()

    def setup_circuit(self):
        # Generate settings
        if not self.settings_path.exists():
            ezkl.gen_settings(self.model_path, self.settings_path)

            # Calibrate settings
            ezkl.calibrate_settings(self.model_path, self.settings_path, target="resources")

        if not self.compiled_path.exists():
            # Compile circuit
            ezkl.compile_circuit(self.model_path, self.compiled_path, self.settings_path)
        
        # Generate proving/verification keys
        if not self.pk_path.exists() or not self.vk_path.exists():
            ezkl.setup(
                self.compiled_path,
                self.pk_path,
                self.vk_path,
                self.settings_path
            )

    def create_proof(self, audio_path):
        # Preprocess audio
        waveform = self.preprocess_audio(audio_path)
        inputs = self.processor(waveform, sampling_rate=self.sample_rate, return_tensors="pt")
        input_values = inputs.input_values.numpy().tolist()
        
        # Create input file
        ezkl.create_input_file(input_values, self.input_path)
        
        # Generate proof
        ezkl.prove(
            self.model_path,
            self.pk_path,
            self.proof_path,
            self.settings_path,
            self.input_path
        )
        
        return self.proof_path.exists()

    def verify_proof(self):
        return ezkl.verify(
            self.proof_path,
            self.vk_path,
            self.settings_path
        )

    def run_workflow(self, audio_path):
        try:
            self.setup_circuit()
            proof_generated = self.create_proof(audio_path)
            verification_status = self.verify_proof()
            
            return {
                "proof_generated": proof_generated,
                "verification_status": verification_status
            }
        except Exception as e:
            return {
                "error": str(e)
            }

# Usage:
import soundfile as sf
audio_info = sf.info("test.wav")
print(audio_info)

pipeline = WhisperEZKL()
results = pipeline.run_workflow("test.wav")



/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-02 11:51:12.248109: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730548272.443734   68596 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730548272.551724   68596 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-02 11:51:13.203345: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical ope

test.wav
samplerate: 44100 Hz
channels: 2
duration: 2.879 s
format: WAV (Microsoft) [WAV]
subtype: Signed 16 bit PCM [PCM_16]


: 

## Model 3 : Silero VAD


In [1]:
import torch
import torchaudio
import ezkl
from pathlib import Path
import json

class SileroEZKL:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.sample_rate = 16000
        self.window_size = 1536
        
        # EZKL paths
        self.model_path = Path("model.onnx")
        self.settings_path = Path("settings.json")
        self.compiled_path = Path("model.ezkl")
        self.pk_path = Path("proving.key")
        self.vk_path = Path("verification.key")
        self.proof_path = Path("proof.json")
        self.input_path = Path("input.json")
        
    def load_model(self):
        try:
            self.model = torch.hub.load(
                repo_or_dir='snakers4/silero-vad', 
                model='silero_vad',
                force_reload=True,
                onnx=True
            )
            
            # Export to ONNX if not exists
            if not self.model_path.exists():
                self.export_to_onnx()
            return True
        except Exception as e:
            print("Model loading failed:", e)
            return False

    def export_to_onnx(self):
        dummy_input = torch.randn(1, self.window_size)
        input_names = ["input_values"]
        output_names = ["last_hidden_state"]
        
        torch.onnx.export(
            self.model,
            dummy_input,
            self.model_path,
            input_names=input_names,
            output_names=output_names,
            dynamic_axes={'input_values': {0: 'batch_size'}}
        )

    def preprocess_audio(self, audio_path):
        waveform, sample_rate = torchaudio.load(audio_path)
        if sample_rate != self.sample_rate:
            resampler = torchaudio.transforms.Resample(sample_rate, self.sample_rate)
            waveform = resampler(waveform)
        return waveform.squeeze()

    def setup_circuit(self):
        if not self.model_path.exists():
            if not self.load_model():
                raise RuntimeError("Failed to load model")
        
        # # Generate settings
        if not self.settings_path.exists():
            ezkl.gen_settings(self.model_path, self.settings_path)

            # Calibrate settings
            ezkl.calibrate_settings(self.model_path, self.settings_path, target="resources")

        if not self.compiled_path.exists():
            # Compile circuit
            ezkl.compile_circuit(self.model_path, self.compiled_path, self.settings_path)
        
        # Generate proving/verification keys
        if not self.pk_path.exists() or not self.vk_path.exists():
            ezkl.setup(
                self.compiled_path,
                self.pk_path,
                self.vk_path,
                self.settings_path
            )

    def create_proof(self, audio_path):
        # Preprocess audio
        waveform = self.preprocess_audio(audio_path)
        inputs = self.processor(waveform, sampling_rate=self.sample_rate, return_tensors="pt")
        input_values = inputs.input_values.numpy().tolist()
        
        # Create input file
        ezkl.create_input_file(input_values, self.input_path)
        
        # Generate proof
        ezkl.prove(
            self.model_path,
            self.pk_path,
            self.proof_path,
            self.settings_path,
            self.input_path
        )
        
        return self.proof_path.exists()

    def verify_proof(self):
        return ezkl.verify(
            self.proof_path,
            self.vk_path,
            self.settings_path
        )

    def run_workflow(self, audio_path):
        self.setup_circuit()
        proof_generated = self.create_proof(audio_path)
        verification_status = self.verify_proof()
        
        return {
            "proof_generated": proof_generated,
            "verification_status": verification_status
        }

In [2]:
pipeline = SileroEZKL()
results = pipeline.run_workflow("test.wav")

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /home/codespace/.cache/torch/hub/master.zip


Model loading failed: 'tuple' object has no attribute 'modules'


RuntimeError: Failed to load model